In [35]:
import wandb
import pandas as pd
# from utils.uploadDataset import upload_dataset_to_wandb
import os
from dotenv import load_dotenv
import json

In [36]:
load_dotenv()
PROJECT_NAME = os.getenv('MAIN_PROJECT_NAME')
VDJDB = os.getenv("VDJDB_ARTIFACT")
MCPAS_TCR = os.getenv("MCPAS_TCR_ARTIFACT")
wandb.init(project=PROJECT_NAME, job_type="data-combination")

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


In [37]:
mcpastcr_artifact = wandb.use_artifact(f"{MCPAS_TCR}:latest")
mcpastcr_table = mcpastcr_artifact.get(f"{MCPAS_TCR}_table.table.json")
mcpastcr_artifact_dir = mcpastcr_artifact.download()
mcpastcr_df = pd.DataFrame(data=mcpastcr_table.data, columns=mcpastcr_table.columns)
mcpastcr_df

wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  


,CDR3.alpha.aa,CDR3.beta.aa,Species,Category,Pathology,Pathology.Mesh.ID,Additional.study.details,Antigen.identification.method,Single.cell,NGS,...,TRAV,TRAJ,TRBV,TRBD,TRBJ,Reconstructed.J.annotation,CDR3.beta.nt,Mouse.strain,PubMed.ID,Remarks
0,nan,CASSDAGANTEVF,Mouse,Pathogens,Lymphocytic choriomeningitis virus (LCMV),D008217,nan,2.1,No,No,...,nan,nan,TRBV8-1,nan,TRBJ1-1,No,nan,P14,1716213,nan
1,nan,CASSDAGAYAEQF,Mouse,Pathogens,Lymphocytic choriomeningitis virus (LCMV),D008217,nan,2.1,No,No,...,nan,nan,TRBV8-1,nan,TRBJ2-1,No,nan,P14,1716213,nan
2,nan,CASSDAGGAAEVF,Mouse,Pathogens,Lymphocytic choriomeningitis virus (LCMV),D008217,nan,2.1,No,No,...,nan,nan,TRBV8-3,nan,TRBJ1-1,No,nan,P14,1716213,nan
3,nan,CASSDAGHSPLYF,Mouse,Pathogens,Lymphocytic choriomeningitis virus (LCMV),D008217,nan,2.1,No,No,...,nan,nan,TRBV8-1,nan,TRBJ1-6,No,nan,P14,1716213,nan
4,nan,CASSDAWGGAEQYF,Mouse,Pathogens,Lymphocytic choriomeningitis virus (LCMV),D008217,nan,2.1,No,No,...,nan,nan,TRBV8-3,nan,TRBJ2-6,No,nan,P14,1716213,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39980,CAGSGAGSYQLTF,CASSLEGQASSYEQYF,Human,Pathogens,Epstein Barr virus (EBV),D020031,nan,1.0,Yes,Yes,...,TRAV25,TRAJ28,nan,nan,nan,nan,nan,nan,35401538,nan
39981,CAGLGAGSYQLTF,CASSLEGQGASYEQYF,Human,Pathogens,Epstein Barr virus (EBV),D020031,nan,1.0,Yes,Yes,...,TRAV25,TRAJ28,nan,nan,nan,nan,nan,nan,35401538,nan
39982,CATEGNSGYSTLTF,CASSSQGGNYGYTF,Human,Pathogens,Epstein Barr virus (EBV),D020031,nan,1.0,Yes,Yes,...,TRAV17*01,TRAJ11*01,nan,nan,nan,nan,nan,nan,35401538,nan
39983,CATEGDSGYSTLTF,CASSYQGGNYGYTF,Human,Pathogens,Epstein Barr virus (EBV),D020031,nan,1.0,Yes,Yes,...,TRAV17*01,TRAJ11*01,nan,nan,nan,nan,nan,nan,35401538,nan


In [38]:
vdjdb_artifact = wandb.use_artifact(f"{VDJDB}:latest")
vdjdb_table = vdjdb_artifact.get(f"{VDJDB}.table.json")
mcpastcr_artifact_dir = vdjdb_artifact.download()
vdjdb_df = pd.DataFrame(data=vdjdb_table.data, columns=vdjdb_table.columns)
vdjdb_df

wandb: Downloading large artifact VDJdb:latest, 79.33MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:0.4
wandb: Downloading large artifact VDJdb:latest, 79.33MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:0.4


,complex.id,Gene,CDR3,V,J,Species,MHC A,MHC B,MHC class,Epitope,Epitope gene,Epitope species,Reference,Method,Meta,CDR3fix,Score
0,1,TRB,CASSYLPGQGDHYSNQPQHF,TRBV13*01,TRBJ1-5*01,HomoSapiens,HLA-B*08,B2M,MHCI,FLKEKGGL,Nef,HIV-1,PMID:15596521,"{""frequency"": """", ""identification"": ""tetramer-...","{""cell.subset"": ""CD8+"", ""clone.id"": """", ""donor...","{""cdr3"": ""CASSYLPGQGDHYSNQPQHF"", ""cdr3_old"": ""...",2
1,0,TRB,CASSFEAGQGFFSNQPQHF,TRBV13*01,TRBJ1-5*01,HomoSapiens,HLA-B*08,B2M,MHCI,FLKEKGGL,Nef,HIV-1,PMID:15596521,"{""frequency"": """", ""identification"": ""tetramer-...","{""cell.subset"": ""CD8+"", ""clone.id"": """", ""donor...","{""cdr3"": ""CASSFEAGQGFFSNQPQHF"", ""cdr3_old"": ""C...",2
2,2,TRB,CASSFEPGQGFYSNQPQHF,TRBV13*01,TRBJ1-5*01,HomoSapiens,HLA-B*08,B2M,MHCI,FLKEKGGL,Nef,HIV-1,PMID:15596521,"{""frequency"": """", ""identification"": ""tetramer-...","{""cell.subset"": ""CD8+"", ""clone.id"": """", ""donor...","{""cdr3"": ""CASSFEPGQGFYSNQPQHF"", ""cdr3_old"": ""C...",2
3,3,TRB,CASSYEPGQVSHYSNQPQHF,TRBV13*01,TRBJ1-5*01,HomoSapiens,HLA-B*08,B2M,MHCI,FLKEKGGL,Nef,HIV-1,PMID:15596521,"{""frequency"": """", ""identification"": ""tetramer-...","{""cell.subset"": ""CD8+"", ""clone.id"": """", ""donor...","{""cdr3"": ""CASSYEPGQVSHYSNQPQHF"", ""cdr3_old"": ""...",2
4,4,TRB,CASSALASLNEQFF,TRBV14*01,TRBJ2-1*01,HomoSapiens,HLA-B*08,B2M,MHCI,FLKEKGGL,Nef,HIV-1,PMID:15596521,"{""frequency"": """", ""identification"": ""tetramer-...","{""cell.subset"": ""CD8+"", ""clone.id"": """", ""donor...","{""cdr3"": ""CASSALASLNEQFF"", ""cdr3_old"": ""CASSAL...",2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84180,30590,TRA,CMDEGGSNYKLTF,TRAV26-1*01,TRAJ53*01,HomoSapiens,HLA-DQA1*05:01,HLA-DQB1*02:01,MHCII,PQPELPYPQPQL,Gluten,Wheat,PMID:33927715,"{""frequency"": """", ""identification"": ""tetramer-...","{""cell.subset"": ""CD4+"", ""clone.id"": """", ""donor...","{""cdr3"": ""CMDEGGSNYKLTF"", ""cdr3_old"": ""CMDEGGS...",0
84181,30591,TRA,CSLYNNNDMRF,TRAV26-1*01,TRAJ43*01,HomoSapiens,HLA-DQA1*05:01,HLA-DQB1*02:01,MHCII,PQPELPYPQPQL,Gluten,Wheat,PMID:33927715,"{""frequency"": """", ""identification"": ""tetramer-...","{""cell.subset"": ""CD4+"", ""clone.id"": """", ""donor...","{""cdr3"": ""CSLYNNNDMRF"", ""cdr3_old"": ""CSLYNNNDM...",0
84182,30592,TRA,CALSTDSWGKLQF,TRAV6*01,TRAJ24*01,HomoSapiens,HLA-DQA1*05:01,HLA-DQB1*02:01,MHCII,PQQPFPQPEQPFP,Gluten,Wheat,PMID:33927715,"{""frequency"": """", ""identification"": ""tetramer-...","{""cell.subset"": ""CD4+"", ""clone.id"": """", ""donor...","{""cdr3"": ""CALSTDSWGKLQF"", ""cdr3_old"": ""CALSTDS...",0
84183,30593,TRA,CAPQGATNKLIF,TRAV12-2*01,TRAJ32*01,HomoSapiens,HLA-DQA1*05:01,HLA-DQB1*02:01,MHCII,PQQPFPQPEQPFP,Gluten,Wheat,PMID:33927715,"{""frequency"": """", ""identification"": ""tetramer-...","{""cell.subset"": ""CD4+"", ""clone.id"": """", ""donor...","{""cdr3"": ""CAPQGATNKLIF"", ""cdr3_old"": ""CAPQGATN...",2


In [41]:
vdjdb_df.columns

Index(['complex.id', 'Gene', 'CDR3', 'V', 'J', 'Species', 'MHC A', 'MHC B',
       'MHC class', 'Epitope', 'Epitope gene', 'Epitope species', 'Reference',
       'Method', 'Meta', 'CDR3fix', 'Score'],
      dtype='object')

In [42]:
mcpastcr_df.columns

Index(['CDR3.alpha.aa', 'CDR3.beta.aa', 'Species', 'Category', 'Pathology',
       'Pathology.Mesh.ID', 'Additional.study.details',
       'Antigen.identification.method', 'Single.cell', 'NGS',
       'Antigen.protein', 'Protein.ID', 'Epitope.peptide', 'Epitope.ID', 'MHC',
       'Tissue', 'T.Cell.Type', 'T.cell.characteristics', 'CDR3.alpha.nt',
       'TRAV', 'TRAJ', 'TRBV', 'TRBD', 'TRBJ', 'Reconstructed.J.annotation',
       'CDR3.beta.nt', 'Mouse.strain', 'PubMed.ID', 'Remarks'],
      dtype='object')